## Simplex Algorithm

In [1]:
import numpy as np
from typing import List, Tuple
np.set_printoptions(precision=6, suppress=True)
from pulp import LpProblem, LpVariable, LpMinimize, LpMaximize, value, PULP_CBC_CMD


### Numpy implementation of the simplex method

In [2]:


TOL = 1e-12   # tolerancia numérica


# --------------------------------------------------------------
#  utilidades de pivoteo
# --------------------------------------------------------------
def _pivot(T: np.ndarray, r: int, s: int) -> None:
    """Pivot in-place sobre el tableau T (fila r, columna s)."""
    T[r] /= T[r, s]
    m, _ = T.shape
    for i in range(m):
        if i != r:
            T[i] -= T[i, s] * T[r]


def _bland_enter(rc: np.ndarray, nonbasic: List[int]) -> int:
    """Devuelve índice column que entra (o -1 si todos rc ≥ 0)."""
    for j, red in zip(nonbasic, rc):
        if red < -TOL:
            return j
    return -1


def _ratio_test(rhs: np.ndarray,
                col: np.ndarray,
                basic: List[int]) -> Tuple[int, float]:
    """Devuelve (fila que sale, theta*).  Bland en empates."""
    theta_min = np.inf
    leave = -1
    for i, (b_i, d_i) in enumerate(zip(rhs, col)):
        if d_i > TOL:
            theta = b_i / d_i
            if theta < theta_min - TOL:
                theta_min, leave = theta, i
            elif abs(theta - theta_min) < TOL and basic[i] < basic[leave]:
                leave = i
    return leave, theta_min


# --------------------------------------------------------------
#  algoritmo principal
# --------------------------------------------------------------
def simplex(A: np.ndarray,
            b: np.ndarray,
            c: np.ndarray,
            verbose: bool = True
            ) -> Tuple[np.ndarray, float, List[int]]:
    """
    Símplex de dos fases (minimización).

    Devuelve:
        x_opt : vector de las n variables originales
        z_opt : valor óptimo de cᵀx
        base  : índices básicos (respecto a las variables originales)
    Lanza ValueError si el problema es infactible o no acotado.
    """
    A = A.astype(float).copy()
    b = b.astype(float).copy()
    c = c.astype(float).copy()
    m, n = A.shape

    # Asegurar b ≥ 0  --------------------------------------------------
    for i in range(m):
        if b[i] < 0:
            A[i] *= -1
            b[i] *= -1

    # Tableau inicial  [A | I | b]  -----------------------------------
    T = np.hstack((A, np.eye(m), b.reshape(-1, 1)))
    total = n + m                  # nº columnas sin RHS

    basic     = list(range(n, n + m))   # artificiales
    nonbasic  = list(range(n))          # originales

    # --------- FASE I: minimizar suma de artificiales ----------------
    cost1 = np.concatenate((np.zeros(n), np.ones(m), [0.0]))
    # hacer canónica
    for i in range(m):
        cost1 -= T[i]
    T = np.vstack((T, cost1))
    phase = 1

    while True:
        # ------------------ impresión -------------------------------
        if verbose:
            z_phase = T[-1, -1] if phase == 1 else -T[-1, -1]
            print(f"Fase {phase}  Base {basic}  z = {z_phase:.6g}")

        # costes reducidos
        rc = T[-1, nonbasic]
        j_enter = _bland_enter(rc, nonbasic)
        if j_enter == -1:                         # óptimo en fase actual
            if phase == 1:
                if T[-1, -1] > 1e-8:
                    raise ValueError("Problema infactible (Fase I > 0).")

                # *Sacar* artificiales que sigan en la base -------------
                for i, var in enumerate(basic):
                    if var >= n:                 # aún artificial
                        row = T[i, :total]
                        # candidato para entrar = primer no-básico viable
                        cand = [j for j in range(n)            # solo originales
                                if j not in basic and abs(row[j]) > TOL]
                        if cand:
                            j_ent = min(cand)                  # Bland
                            _pivot(T, i, j_ent)
                            nonbasic.remove(j_ent)
                            nonbasic.append(var)
                            basic[i] = j_ent
                        else:
                            # fila artificial redundant → dejarla (se elimina columna)
                            pass

                # eliminar columnas artificiales
                keep = list(range(n)) + [-1]
                T = T[:, keep]
                total = n
                # ---- CORRECCIÓN: limpiar la lista de no básicos -----
                nonbasic = sorted(j for j in nonbasic if j < n)
                # quitar fila objetivo de Fase I
                T = T[:-1, :]

                # construir fila de costes original  -c + c_B B^{-1}A
                cost2 = np.concatenate((-c, [0.0]))
                for i, var in enumerate(basic):
                    cost2 += c[var] * T[i]
                T = np.vstack((T, -cost2))
                phase = 2
                continue     # vuelve al while con la Fase II

            else:            # fase 2 óptima
                x = np.zeros(n)
                x[basic] = T[:-1, -1]            # valores básicos
                z = -T[-1, -1]
                return x, z, basic

        # ------------------ ratio test -------------------------------
        col = T[:-1, j_enter]
        rhs = T[:-1, -1]
        i_leave, _ = _ratio_test(rhs, col, basic)
        if i_leave == -1:
            raise ValueError("Problema no acotado.")

        _pivot(T, i_leave, j_enter)

        # actualizar listas
        var_leave = basic[i_leave]
        nonbasic[nonbasic.index(j_enter)] = var_leave
        basic[i_leave] = j_enter





## Pulp Implementation

In [3]:


def solve_with_pulp(A: np.ndarray,
                    b: np.ndarray,
                    c: np.ndarray,
                    sense: str = "min",
                    msg:bool=False
                   ):
    """
    Resuelve  min/max c^T x  s.a. A x = b, x >= 0   con PuLP.
    Parámetro sense debe ser "min" o "max".
    """
    m, n = A.shape

    # Seleccionar sentido
    if sense == "min":
        direction = LpMinimize
    elif sense == "max":
        direction = LpMaximize
    else:
        raise ValueError("sense debe ser 'min' o 'max'")

    # Crear problema
    prob = LpProblem("LP", direction)

    # Variables (>=0)
    x = [LpVariable(f"x{i+1}", lowBound=0) for i in range(n)]

    # Función objetivo
    prob += sum(c[j] * x[j] for j in range(n))

    # Restricciones Ax = b
    for i in range(m):
        prob += sum(A[i, j] * x[j] for j in range(n)) == b[i]
    
    solver = PULP_CBC_CMD(msg=msg)
    prob.solve(solver)

    x_val = np.array([value(v) for v in x])
    z_val = value(prob.objective)
    return x_val, z_val


### Check the rank of A

In [4]:
def matrix_rank(A: np.ndarray, tol: float = 1e-12) -> int:
    """
    Devuelve el rango numérico de la matriz A (ndarray) usando SVD.

    Parámetros
    ----------
    A   : array_like, shape (m, n)
    tol : tolerancia relativa/absoluta para considerar un valor singular “cero”.
          Por defecto 1·10⁻¹².

    Retorna
    -------
    r   : int
        Número de valores singulares mayores que tol · σ_max.
    """
    # valores singulares de A
    s = np.linalg.svd(A, compute_uv=False)
    if s.size == 0:        # matriz vacía
        return 0
    # umbral: tol * σ_max
    thresh = tol * s[0]
    return int(np.sum(s > thresh))

## Solve the example

In [5]:
A = np.array([[1, 1, 1, 0],
              [2, 1, 0, 1]], dtype=float)
b = np.array([4, 5], dtype=float)
c = np.array([-3, -2, 0, 0], dtype=float)      # minimizar cᵀx
print("A =\n", A)      # nota el \n
print("b =", b, "\n")  # \n opcional para dejar línea en blanco
print("c =", c)

A =
 [[1. 1. 1. 0.]
 [2. 1. 0. 1.]]
b = [4. 5.] 

c = [-3. -2.  0.  0.]


In [6]:
A.shape

(2, 4)

In [7]:
matrix_rank(A)

2

In [8]:
x_opt, z_opt, final_basis = simplex(A, b, c)

print("\nSolución óptima:", x_opt)
print("Costo óptimo   :", z_opt)
print("Base final     :", final_basis)

Fase 1  Base [4, 5]  z = -9
Fase 1  Base [4, 0]  z = -1.5
Fase 1  Base [1, 0]  z = 0
Fase 2  Base [1, 0]  z = -9

Solución óptima: [1. 3. 0. 0.]
Costo óptimo   : -9.0
Base final     : [1, 0]


In [9]:
x_opt, z_opt = solve_with_pulp(A, b, c, sense="min")
print("Solución óptima =", x_opt)
print("Costo óptimo =", z_opt)

Solución óptima = [1. 3. 0. 0.]
Costo óptimo = -9.0


## Bigger example

In [21]:
A = np.array([
    [5, 1, 4, 4, 4, -2, 4, 3, 52, 1],
    [1, 5, -3, 2, 1, 2, -2, 14, 2, -5],
    [4, -1, 4, 1, -1, 4, -214, 2, 4, 4],
    [4, -11, 2, 2, 2, 1, 0, 5, 14, 4],
    [3, 5, 3, 1, 1, -5, 11, 5, 2, 5]
], dtype=float)

b = np.array([33, 24, 28, 28, 31], dtype=float)   #  = A @ np.ones(10)

# función objetivo  min c^T x
c = np.array([9, 2, 2, 8, 4, 7, 8, 3, 1, 4], dtype=float)

In [22]:
matrix_rank(A)

5

In [23]:
A.shape

(5, 10)

### Pulp solution

In [25]:
x_opt, z_opt = solve_with_pulp(A, b, c, sense="min")
print("Solución óptima=", x_opt)
print("Costo óptimo =", z_opt)

Solución óptima= [0.       0.       4.624128 0.       0.       0.168269 0.       2.899078
 0.105583 0.652482]
Costo óptimo = 21.838880460000002


### Numpy Solution

In [26]:
# La base identidad inicial son las columnas 2 y 3 (índices 2,3)
x_opt, z_opt, final_basis = simplex(A, b, c)

print("\nSolución óptima:", x_opt)
print("Costo óptimo   :", z_opt)
print("Base final     :", final_basis)

Fase 1  Base [10, 11, 12, 13, 14]  z = -144
Fase 1  Base [0, 11, 12, 13, 14]  z = -31.8
Fase 1  Base [0, 11, 5, 13, 14]  z = -29.8571
Fase 1  Base [0, 11, 5, 6, 14]  z = -29.4257
Fase 1  Base [0, 1, 5, 6, 14]  z = -26.0772
Fase 1  Base [0, 1, 5, 7, 14]  z = -8.25886
Fase 1  Base [0, 1, 2, 7, 14]  z = -2.21821
Fase 1  Base [0, 3, 2, 7, 14]  z = -2.02256
Fase 1  Base [0, 4, 2, 7, 14]  z = -1.72727
Fase 1  Base [0, 4, 2, 7, 13]  z = -0.904762
Fase 1  Base [0, 6, 2, 7, 13]  z = -0.385214
Fase 1  Base [0, 6, 2, 7, 5]  z = -1.57652e-14
Fase 2  Base [0, 6, 2, 7, 5]  z = 26.6811
Fase 2  Base [1, 6, 2, 7, 5]  z = 26.1317
Fase 2  Base [3, 6, 2, 7, 5]  z = 25.9826
Fase 2  Base [4, 6, 2, 7, 5]  z = 25.807
Fase 2  Base [8, 6, 2, 7, 5]  z = 25.7192
Fase 2  Base [8, 9, 2, 7, 5]  z = 21.8389

Solución óptima: [0.       0.       4.624128 0.       0.       0.168269 0.       2.899078
 0.105583 0.652482]
Costo óptimo   : 21.83888048411497
Base final     : [8, 9, 2, 7, 5]
